<a href="https://colab.research.google.com/github/iust-deep-learning/tensorflow-2-tutorial/blob/master/part_02_tensors_and_basic_ops/solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part #2: Tensors and Basic Operations


TensorFlow 2.0 Tutorial by IUST

*   Last Update: Jan 2020
*   Official Page: https://github.com/iust-deep-learning/tensorflow-2-tutorial





---




Please run the following cell before going through the rest of the tutorial.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np

from pprint import pprint

TensorFlow 2.x selected.


### Exercise #1: Prime numbers diff
We have a very special vector, where the `i`th element is equal to the absolute difference of `i`th prime number squared and `i+1`th prime number squared. For example,  the 1st element in this vector is $|2^2 - 3^2| = 5$. Create this vector using TensorFlow operations.


In [0]:
prime_numbers = tf.constant([2, 3, 5, 7, 11, 13, 17, 19, 23])

# Put your answer here, complete the definition of Tensor `diff`
# You may want to use tf.square and tf.abs
diffs = tf.zeros_like(prime_numbers)

assert tf.math.reduce_all(diffs == tf.constant([5,  16,  24,  72,  48, 120,  72, 168, 312])).numpy()
print("Passed!")

Solution:



In [0]:
prime_numbers = tf.constant([2, 3, 5, 7, 11, 13, 17, 19, 23])

shifted_prime_numbers = tf.concat([prime_numbers[1:], [29]], axis=0)

# Put your answer here, complete the definition of Tensor `diff`
diffs = tf.abs(tf.square(prime_numbers) - tf.square(shifted_prime_numbers))

assert tf.math.reduce_all(diffs == tf.constant([5,  16,  24,  72,  48, 120,  72, 168, 312])).numpy()
print("Passed")

### Exercise #2: Simulate 10 throwings of one die and one coin.
Perform this simulation, and store its result in a `[10, 3]` shaped int32 tensor. Each row belongs to one simulation (that is, one roll of a six-sided die and one flip of a coin). The definition of each column is as follows:
*   Column 1: the result of throwing the coin.
*   Column 2: the result of throwing the die.
*   Column 3: if we have head and the roll's result is bigger than 3, then it should be `1`, otherwise it should be `0`)

For example, one of the rows might be something like  `[1, 4, 1]`

In [0]:
# Write your answer here. 
# Complete the implementation of `simulation_result` tensor
# You may need to use tf.math.greater and tf.math.equal 

simulation_result = tf.zeros(shape=(10, 3), dtype=tf.int32)
simulation_result

Solution

In [0]:
coin_flip = tf.random.uniform([10, 1], maxval=2, dtype=tf.int32)
die_roll = tf.random.uniform([10, 1], minval=1, maxval=7, dtype=tf.int32)

die_is_bigger_than = tf.cast(tf.math.greater(die_roll, 3), tf.int32)
success = tf.cast(tf.math.equal(die_is_bigger_than + coin_flip, 2), tf.int32)

simulation_result = tf.concat([coin_flip, die_roll, success], axis=1)
simulation_result

<tf.Tensor: shape=(10, 3), dtype=int32, numpy=
array([[0, 1, 0],
       [0, 5, 0],
       [0, 4, 0],
       [1, 5, 1],
       [1, 5, 1],
       [1, 2, 0],
       [1, 2, 0],
       [1, 4, 1],
       [1, 3, 0],
       [0, 4, 0]], dtype=int32)>

### Exercise #3: Normalized Euclidean Distance 

Suppose that we have two sets of d-dimensional vectors. Our goal is to calculate the normalized euclidean distance between each vector of these two sets. That is, given $S_1 \in R^{m \times d}$ and $S_1 \in R^{n \times d}$, we want to calculate the $X \in R^{m \times n}$. Euclidean distance between two vector V and W is calculated as follows:

$$
dist = \sqrt{(V_1 - W_1)^2 + ... +\:(W_d - W_d)^2}
$$

Please note that we want to calculate the normalized distance, which is within the [0, 1] range. Therefore, you should normalize the similarity scores across each row.

In [0]:
def euclidean_norm_distance(v, w):
  # Write your answer here. 
  raise NotImplemented()

In [0]:
t1 = tf.constant([[-1.8897635 ,  0.7396171 ,  0.4683413 ,  2.35642   , -0.8153529 ],
       [ 1.3100415 ,  0.6090922 ,  0.70573515,  0.07053893, -0.20450763],
       [-0.14293706, -0.94566655,  0.41517866,  0.9539284 , -0.9522885 ]])
t2 = tf.constant([[ 0.4980808 ,  0.12677321, -1.6533084 ,  1.2168828 ,  0.351612  ],
       [-0.35999015, -1.013327  , -1.4144444 ,  0.83520454,  1.4889846 ]])

answer = tf.constant([[0.4718873, 0.5281127 ],
 [0.43739235, 0.5626077 ],
 [0.47395885, 0.52604115]])

assert np.allclose(euclidean_norm_distance(t1, t2).numpy(), answer)
print("Passed!")

Passed!


Solution

In [0]:
def euclidean_norm_distance(v, w):
  # v = (3, 5)
  # w = (2, 5)
  n = tf.shape(v)[0] # 3
  m = tf.shape(w)[0] # 2

  v = tf.tile(tf.expand_dims(v, 1), [1, m, 1]) # (n, m, d)
  w = tf.tile(tf.expand_dims(w, 1), [1, n, 1]) # (m, n, d)
  w = tf.transpose(w, [1, 0, 2]) # (n, m, d)

  distances = (v - w) # (n, m, d)
  distances = distances ** 2 # (n, m, d)
  distances = tf.math.reduce_sum(distances, axis=2) # (n, m)
  distances = tf.math.sqrt(distances) # (n, m)

  sum_distances = tf.reshape(tf.math.reduce_sum(distances, axis=1), [-1, 1]) # (n, 1)
  distances = distances / sum_distances # (n, m)

  return distances